# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"
### Задание 1.
### Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

### Решение

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

# что ищем в статьях 
KEYWORDS = ['python', 'парсинг']
URL = 'https://habr.com/ru/search/'
params = {
    'target_type': 'posts',
    'order_by' : 'relevance',
    'q': KEYWORDS[0] +' '+ KEYWORDS[1],
    'flow': []
    }

headers = {
    'user_agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
    }
req = requests.get(URL, params=params, headers=headers)
req.text

'<!DOCTYPE html>\n<html lang="ru" class="no-js">\n  <head>\n    <meta http-equiv="content-type" content="text/html; charset=utf-8" />\n<meta content=\'width=1024\' name=\'viewport\'>\n<title>Результаты поиска по запросу «python парсинг» / Хабр</title>\n\n\n\n\n\n<meta name=\'yandex-verification\' content=\'71593b225aeafc4e\' />\n<meta name=\'referrer\' content=\'unsafe-url\' />\n<meta name="pocket-site-verification" content="ed24b2b9721edf0a282c5b4a3232c4" />\n<meta name="biu" content="https://dr.habracdn.net/habr/602f962a/images/">\n\n<style type="text/css">\n  @font-face{font-family:\'Fira Sans\';font-style:normal;font-weight:500;src:url(https://dr.habracdn.net/habr/602f962a/fonts/FiraSans/firaSans-medium.eot);src:local("Fira Sans Medium"),local("FiraSans-Medium"),url(https://dr.habracdn.net/habr/602f962a/fonts/FiraSans/firaSans-medium.eot?#iefix) format("embedded-opentype"),url(https://dr.habracdn.net/habr/602f962a/fonts/FiraSans/firaSans-medium.woff2) format("woff2"),url(https://dr

In [2]:
soup = BeautifulSoup(req.text, 'html.parser')
soup

<!DOCTYPE html>

<html class="no-js" lang="ru">
<head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="width=1024" name="viewport"/>
<title>Результаты поиска по запросу «python парсинг» / Хабр</title>
<meta content="71593b225aeafc4e" name="yandex-verification">
<meta content="unsafe-url" name="referrer"/>
<meta content="ed24b2b9721edf0a282c5b4a3232c4" name="pocket-site-verification"/>
<meta content="https://dr.habracdn.net/habr/602f962a/images/" name="biu"/>
<style type="text/css">
  @font-face{font-family:'Fira Sans';font-style:normal;font-weight:500;src:url(https://dr.habracdn.net/habr/602f962a/fonts/FiraSans/firaSans-medium.eot);src:local("Fira Sans Medium"),local("FiraSans-Medium"),url(https://dr.habracdn.net/habr/602f962a/fonts/FiraSans/firaSans-medium.eot?#iefix) format("embedded-opentype"),url(https://dr.habracdn.net/habr/602f962a/fonts/FiraSans/firaSans-medium.woff2) format("woff2"),url(https://dr.habracdn.net/habr/602f962a/fonts/FiraSans/fir

In [3]:
all_posts = soup.find_all('article', class_='post')
all_posts

[<article class="post post_preview" lang="ru">
 <header class="post__meta">
 <a class="post__user-info user-info" href="https://habr.com/ru/users/talurus/" title="Автор публикации">
 <img class="user-info__image-pic user-info__image-pic_small" height="24" src="//habrastorage.org/getpro/habr/avatars/b20/97f/c05/b2097fc054023f1b4bc51dc3a8502ee6.jpg" width="24"/>
 <span class="user-info__nickname user-info__nickname_small">talurus</span>
 </a>
 <span class="post__time">14 июня 2011 в 19:24</span>
 </header>
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/post/121815/"><em class="searched-item">Парсинг</em> на Pуthon. Как собрать архив Голубятен</a>
 </h2>
 <ul class="post__hubs inline-list">
 <li class="inline-list__item inline-list__item_hub">
 <a class="inline-list__item-link hub-link" href="https://habr.com/ru/hub/python/" onclick="if (typeof ga === 'function') { ga('send', 'event', 'hub', 'feed page', 'Python'); }" rel="nofollow" title="Вы не подписаны

In [20]:
# добираемся до даты
datka = list(map(lambda x: x.find('span', class_='post__time'), all_posts))
datka

[<span class="post__time">14 июня 2011 в 19:24</span>,
 <span class="post__time">1 октября 2020 в 17:17</span>,
 <span class="post__time">31 июля 2020 в 10:00</span>,
 <span class="post__time">6 марта 2015 в 17:40</span>,
 <span class="post__time">28 октября 2014 в 10:00</span>,
 <span class="post__time">7 мая 2019 в 14:51</span>,
 <span class="post__time">21 марта 2020 в 12:30</span>,
 <span class="post__time">2 сентября 2011 в 14:37</span>,
 <span class="post__time">18 декабря 2013 в 10:12</span>,
 <span class="post__time">3 октября 2014 в 12:03</span>,
 <span class="post__time">3 марта 2016 в 16:39</span>,
 <span class="post__time">23 июля 2020 в 15:17</span>,
 <span class="post__time">27 июня 2019 в 17:03</span>,
 <span class="post__time">2 мая 2020 в 23:42</span>,
 <span class="post__time">3 апреля 2019 в 15:21</span>,
 <span class="post__time">26 февраля 2018 в 09:23</span>,
 <span class="post__time">8 ноября 2018 в 14:00</span>,
 <span class="post__time">1 марта 2019 в 15:00</sp

In [10]:
# добираемся до текста со ссылкой
articles = list(map(lambda x: x.find('a', class_='post__title_link'), all_posts))
articles

[<a class="post__title_link" href="https://habr.com/ru/post/121815/"><em class="searched-item">Парсинг</em> на Pуthon. Как собрать архив Голубятен</a>,
 <a class="post__title_link" href="https://habr.com/ru/post/521646/">Как выбрать решение для <em class="searched-item">парсинга</em> сайтов: классификация и большой обзор программ, сервисов и фреймворков</a>,
 <a class="post__title_link" href="https://habr.com/ru/post/512208/">Как проанализировать рынок фотостудий с помощью <em class="searched-item">Python</em> (2/3). База данных</a>,
 <a class="post__title_link" href="https://habr.com/ru/post/252379/">Веб-<em class="searched-item">парсинг</em> на Ruby</a>,
 <a class="post__title_link" href="https://habr.com/ru/post/241670/">Вглубь Pyparsing: парсим единицы измерения на <em class="searched-item">Python</em></a>,
 <a class="post__title_link" href="https://habr.com/ru/post/450834/"><em class="searched-item">Парсинг</em> сайтов — а это вообще легально в России?</a>,
 <a class="post__title_

In [11]:
# возьмём ссылки
reference_list = list(map(lambda x: x.get('href'), articles))
reference_list

['https://habr.com/ru/post/121815/',
 'https://habr.com/ru/post/521646/',
 'https://habr.com/ru/post/512208/',
 'https://habr.com/ru/post/252379/',
 'https://habr.com/ru/post/241670/',
 'https://habr.com/ru/post/450834/',
 'https://habr.com/ru/company/piter/blog/493424/',
 'https://habr.com/ru/post/127584/',
 'https://habr.com/ru/post/205944/',
 'https://habr.com/ru/post/239081/',
 'https://habr.com/ru/company/pt/blog/210772/',
 'https://habr.com/ru/post/504900/',
 'https://habr.com/ru/company/otus/blog/457942/',
 'https://habr.com/ru/post/500162/',
 'https://habr.com/ru/post/446488/',
 'https://habr.com/ru/post/349860/',
 'https://habr.com/ru/company/ods/blog/348260/',
 'https://habr.com/ru/company/otus/blog/442252/',
 'https://habr.com/ru/post/113065/',
 'https://habr.com/ru/post/262991/']

In [21]:
#данным действием создаём датафрейм и помещаем туда информацию дату - заголовок - ссылку из статей по поиску
habr_news = pd.DataFrame()
for i in range(len(articles)):
    rows = {'date' : datka[i].text, 'header' : articles[i].text, 'link' : reference_list[i]}
    habr_news = pd.concat([habr_news, pd.DataFrame([rows])])
habr_news.reset_index(drop=True)

,date,header,link
0,14 июня 2011 в 19:24,Парсинг на Pуthon. Как собрать архив Голубятен,https://habr.com/ru/post/121815/
1,1 октября 2020 в 17:17,Как выбрать решение для парсинга сайтов: класс...,https://habr.com/ru/post/521646/
2,31 июля 2020 в 10:00,Как проанализировать рынок фотостудий с помощь...,https://habr.com/ru/post/512208/
3,6 марта 2015 в 17:40,Веб-парсинг на Ruby,https://habr.com/ru/post/252379/
4,28 октября 2014 в 10:00,Вглубь Pyparsing: парсим единицы измерения на ...,https://habr.com/ru/post/241670/
5,7 мая 2019 в 14:51,Парсинг сайтов — а это вообще легально в России?,https://habr.com/ru/post/450834/
6,21 марта 2020 в 12:30,Реализуем преобразования кода на Python,https://habr.com/ru/company/piter/blog/493424/
7,2 сентября 2011 в 14:37,Grab — python библиотека для парсинга сайтов,https://habr.com/ru/post/127584/
8,18 декабря 2013 в 10:12,Python на Хабре,https://habr.com/ru/post/205944/
9,3 октября 2014 в 12:03,Парсим на Python: Pyparsing для новичков,https://habr.com/ru/post/239081/


### Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [17]:
# что ищем в статьях 
KEYWORDS = ['python', 'парсинг']
URL = 'https://habr.com/ru/search/'
params = {
    'target_type': 'posts',
    'order_by' : 'relevance',
    'q': KEYWORDS[0] +' '+ KEYWORDS[1],
    'flow': []
    }

headers = {
    'user_agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
    }
req2 = requests.get(URL, params=params, headers=headers)
req2.text


'<!DOCTYPE html>\n<html lang="ru" class="no-js">\n  <head>\n    <meta http-equiv="content-type" content="text/html; charset=utf-8" />\n<meta content=\'width=1024\' name=\'viewport\'>\n<title>Результаты поиска по запросу «python парсинг» / Хабр</title>\n\n\n\n\n\n<meta name=\'yandex-verification\' content=\'71593b225aeafc4e\' />\n<meta name=\'referrer\' content=\'unsafe-url\' />\n<meta name="pocket-site-verification" content="ed24b2b9721edf0a282c5b4a3232c4" />\n<meta name="biu" content="https://dr.habracdn.net/habr/602f962a/images/">\n\n<style type="text/css">\n  @font-face{font-family:\'Fira Sans\';font-style:normal;font-weight:500;src:url(https://dr.habracdn.net/habr/602f962a/fonts/FiraSans/firaSans-medium.eot);src:local("Fira Sans Medium"),local("FiraSans-Medium"),url(https://dr.habracdn.net/habr/602f962a/fonts/FiraSans/firaSans-medium.eot?#iefix) format("embedded-opentype"),url(https://dr.habracdn.net/habr/602f962a/fonts/FiraSans/firaSans-medium.woff2) format("woff2"),url(https://dr

In [19]:
soup2 = BeautifulSoup(req.text, 'html.parser')
soup2

<!DOCTYPE html>

<html class="no-js" lang="ru">
<head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="width=1024" name="viewport"/>
<title>Результаты поиска по запросу «python парсинг» / Хабр</title>
<meta content="71593b225aeafc4e" name="yandex-verification">
<meta content="unsafe-url" name="referrer"/>
<meta content="ed24b2b9721edf0a282c5b4a3232c4" name="pocket-site-verification"/>
<meta content="https://dr.habracdn.net/habr/602f962a/images/" name="biu"/>
<style type="text/css">
  @font-face{font-family:'Fira Sans';font-style:normal;font-weight:500;src:url(https://dr.habracdn.net/habr/602f962a/fonts/FiraSans/firaSans-medium.eot);src:local("Fira Sans Medium"),local("FiraSans-Medium"),url(https://dr.habracdn.net/habr/602f962a/fonts/FiraSans/firaSans-medium.eot?#iefix) format("embedded-opentype"),url(https://dr.habracdn.net/habr/602f962a/fonts/FiraSans/firaSans-medium.woff2) format("woff2"),url(https://dr.habracdn.net/habr/602f962a/fonts/FiraSans/fir

In [23]:
all_posts2 = soup.find_all('article', class_='post')
all_posts2

[<article class="post post_preview" lang="ru">
 <header class="post__meta">
 <a class="post__user-info user-info" href="https://habr.com/ru/users/talurus/" title="Автор публикации">
 <img class="user-info__image-pic user-info__image-pic_small" height="24" src="//habrastorage.org/getpro/habr/avatars/b20/97f/c05/b2097fc054023f1b4bc51dc3a8502ee6.jpg" width="24"/>
 <span class="user-info__nickname user-info__nickname_small">talurus</span>
 </a>
 <span class="post__time">14 июня 2011 в 19:24</span>
 </header>
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/post/121815/"><em class="searched-item">Парсинг</em> на Pуthon. Как собрать архив Голубятен</a>
 </h2>
 <ul class="post__hubs inline-list">
 <li class="inline-list__item inline-list__item_hub">
 <a class="inline-list__item-link hub-link" href="https://habr.com/ru/hub/python/" onclick="if (typeof ga === 'function') { ga('send', 'event', 'hub', 'feed page', 'Python'); }" rel="nofollow" title="Вы не подписаны

In [26]:
# добираемся до текста со ссылкой
articles2 = list(map(lambda x: x.find('a', class_='post__title_link'), all_posts2))
articles2

[<a class="post__title_link" href="https://habr.com/ru/post/121815/"><em class="searched-item">Парсинг</em> на Pуthon. Как собрать архив Голубятен</a>,
 <a class="post__title_link" href="https://habr.com/ru/post/521646/">Как выбрать решение для <em class="searched-item">парсинга</em> сайтов: классификация и большой обзор программ, сервисов и фреймворков</a>,
 <a class="post__title_link" href="https://habr.com/ru/post/512208/">Как проанализировать рынок фотостудий с помощью <em class="searched-item">Python</em> (2/3). База данных</a>,
 <a class="post__title_link" href="https://habr.com/ru/post/252379/">Веб-<em class="searched-item">парсинг</em> на Ruby</a>,
 <a class="post__title_link" href="https://habr.com/ru/post/241670/">Вглубь Pyparsing: парсим единицы измерения на <em class="searched-item">Python</em></a>,
 <a class="post__title_link" href="https://habr.com/ru/post/450834/"><em class="searched-item">Парсинг</em> сайтов — а это вообще легально в России?</a>,
 <a class="post__title_

In [34]:
# возьмём ссылки
reference_list2 = list(map(lambda x: x.get('href'), articles2))
reference_list2

['https://habr.com/ru/post/121815/',
 'https://habr.com/ru/post/521646/',
 'https://habr.com/ru/post/512208/',
 'https://habr.com/ru/post/252379/',
 'https://habr.com/ru/post/241670/',
 'https://habr.com/ru/post/450834/',
 'https://habr.com/ru/company/piter/blog/493424/',
 'https://habr.com/ru/post/127584/',
 'https://habr.com/ru/post/205944/',
 'https://habr.com/ru/post/239081/',
 'https://habr.com/ru/company/pt/blog/210772/',
 'https://habr.com/ru/post/504900/',
 'https://habr.com/ru/company/otus/blog/457942/',
 'https://habr.com/ru/post/500162/',
 'https://habr.com/ru/post/446488/',
 'https://habr.com/ru/post/349860/',
 'https://habr.com/ru/company/ods/blog/348260/',
 'https://habr.com/ru/company/otus/blog/442252/',
 'https://habr.com/ru/post/113065/',
 'https://habr.com/ru/post/262991/']

In [35]:

req_into_post = requests.get('https://habr.com/ru/post/121815/')
req_into_post_soup = BeautifulSoup(req_into_post.text, 'html.parser')
req_into_post_soup

<!DOCTYPE html>

<html class="no-js" lang="ru">
<head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="width=1024" name="viewport"/>
<title>Парсинг на Pуthon. Как собрать архив Голубятен / Хабр</title>
<meta content="Статья описывает разработку скрипта на языке Python. Скрипт выполняет парсинг HTML-кода, составление списка материалов сайта, скачивания статей и предварительную очистку текста статьи от..." name="description">
<meta content="Python, парсинг, urllib, lxml, xpath, configobj" name="keywords"/>
<meta content="444736788986613" property="fb:app_id"/>
<meta content="article" property="og:type"/>
<meta content="472597926099084" property="fb:pages"/>
<meta content="https://habr.com/ru/post/121815/" property="og:url"/>
<meta content="Парсинг на Pуthon. Как собрать архив Голубятен" property="og:title"/>
<meta content="https://habr.com/share/publication/121815/fe04db0bb8e8da1fc80879c70a812ca9/?v=1" property="og:image"/>
<meta content="1200" propert

In [44]:
# создаём датафрейм и помещаем туда информацию <дата> - <заголовок> - <ссылка> - <текст_статьи> из статей по поиску 
# используя цикл for пробегаясь по ссылкам
habr_news2 = pd.DataFrame()

for i in range(len(reference_list2)):
    URL2 = reference_list2[i]
    req_into_post = requests.get(URL2)
    time.sleep(0.33)
    req_into_post_soup = BeautifulSoup(req_into_post.text, 'html.parser')
    title2 = req_into_post_soup.find('span', class_ = 'post__title-text').text
    datka2 = req_into_post_soup.find('span', class_ = 'post__time').text
    post_text = req_into_post_soup.find('div', class_= 'post__text').text
    rows2 = {'date' : datka2, 'header' : title2, 'link' : reference_list2[i], 'text' : post_text}
    habr_news2 = pd.concat([habr_news2, pd.DataFrame([rows2])])
habr_news2.reset_index(drop=True)

,date,header,link,text
0,14 июня 2011 в 19:24,Парсинг на Pуthon. Как собрать архив Голубятен,https://habr.com/ru/post/121815/,Статья описывает разработку скрипта на языке P...
1,1 октября 2020 в 17:17,Как выбрать решение для парсинга сайтов: класс...,https://habr.com/ru/post/521646/,Парсинг или как его еще иногда называют web sc...
2,31 июля 2020 в 10:00,Как проанализировать рынок фотостудий с помощь...,https://habr.com/ru/post/512208/,В предыдущей статье в рамках коммерческого про...
3,6 марта 2015 в 17:40,Веб-парсинг на Ruby,https://habr.com/ru/post/252379/,"\nЭто перевод статьи «Web Scraping with Ruby»,..."
4,28 октября 2014 в 10:00,Вглубь Pyparsing: парсим единицы измерения на ...,https://habr.com/ru/post/241670/,В прошлой статье мы познакомились с удобной би...
5,7 мая 2019 в 14:51,Парсинг сайтов — а это вообще легально в России?,https://habr.com/ru/post/450834/,По одному из определений парсинг есть синтакси...
6,21 марта 2020 в 12:30,Реализуем преобразования кода на Python,https://habr.com/ru/company/piter/blog/493424/,"Привет, Хабр.\n\r\nСегодня мы предлагаем вам п..."
7,2 сентября 2011 в 14:37,Grab — python библиотека для парсинга сайтов,https://habr.com/ru/post/127584/,"Лет пять-шесть назад, когда я ещё программиров..."
8,18 декабря 2013 в 10:12,Python на Хабре,https://habr.com/ru/post/205944/,"Некоторое время назад, в силу определенных при..."
9,3 октября 2014 в 12:03,Парсим на Python: Pyparsing для новичков,https://habr.com/ru/post/239081/,Парсинг (синтаксический анализ) представляет с...


### Задание 2.
### Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.
    
### Решение

In [52]:
import json
EMAIL = ['xxx@x.ru', 'yyy@y.com']
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

headers= {
    'Vaar-Header-App-Product' : 'hackcheck-web-avast',
    'Vaar-Version' : '0'
         }

dictionary_emails = {
    'emailAddresses' : EMAIL
}

In [50]:
request = requests.post(URL, json=dictionary_emails, headers=headers)
request.text

'{"breaches":{"17110":{"breachId":17110,"site":"azcentral.com","recordsCount":705538,"description":"At an unconfirmed date, online Arizona newspaper \\"AZ Central\\" was allegedly breached. The stolen data contains passwords and email addresses. This breach is being publicly shared on the internet.","publishDate":"2020-01-03T00:00:00Z","statistics":{"usernames":0,"passwords":702971,"emails":705538}},"17670":{"breachId":17670,"site":"wishbone.io","recordsCount":37475476,"description":"In January 2020, the online poll website Wishbone was allegedly breached. The stolen data contains usernames, passwords, email addresses and additional personal information. This breach is being privately shared on the internet.","publishDate":"2020-05-28T00:00:00Z","statistics":{"usernames":31111127,"passwords":10648734,"emails":10234249}},"13094":{"breachId":13094,"site":"myheritage.com","recordsCount":110041653,"description":"In October 2017, a customer database belonging to online genealogy platform My

In [70]:
dict_response = json.loads(request.text)

In [88]:
dict_response['summary']

{'xxx@x.ru': {'breaches': [3176, 12, 3, 3164, 2961, 15]},
 'yyy@y.com': {'breaches': [17110,
   17670,
   13094,
   37177,
   16768,
   41,
   2,
   3587,
   17009,
   3,
   13662,
   3520,
   15,
   3669]}}

In [75]:
dict_response['breaches']

{'17110': {'breachId': 17110,
  'site': 'azcentral.com',
  'recordsCount': 705538,
  'description': 'At an unconfirmed date, online Arizona newspaper "AZ Central" was allegedly breached. The stolen data contains passwords and email addresses. This breach is being publicly shared on the internet.',
  'publishDate': '2020-01-03T00:00:00Z',
  'statistics': {'usernames': 0, 'passwords': 702971, 'emails': 705538}},
 '17670': {'breachId': 17670,
  'site': 'wishbone.io',
  'recordsCount': 37475476,
  'description': 'In January 2020, the online poll website Wishbone was allegedly breached. The stolen data contains usernames, passwords, email addresses and additional personal information. This breach is being privately shared on the internet.',
  'publishDate': '2020-05-28T00:00:00Z',
  'statistics': {'usernames': 31111127,
   'passwords': 10648734,
   'emails': 10234249}},
 '13094': {'breachId': 13094,
  'site': 'myheritage.com',
  'recordsCount': 110041653,
  'description': 'In October 2017, 

In [134]:
# <почта> - <дата утечки> - <источник утечки> - <описание утечки>
hack_list = pd.DataFrame()

for mail, breId in dict_response['summary'].items():

    for index in breId['breaches']:
        index = str(index)
        site = dict_response['breaches'][index]['site']
        description = dict_response['breaches'][index]['description']
        date = dict_response['breaches'][index]['publishDate']
        rows = {'mail' : mail, 'date' : date, 'site' : site, 'description' :  description}
        hack_list = pd.concat([hack_list, pd.DataFrame([rows])])
        
hack_list.reset_index(drop=True)  

,mail,date,site,description
0,xxx@x.ru,2017-02-14T00:00:00Z,parapa.mail.ru,"In July and August 2016, two criminals execute..."
1,xxx@x.ru,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
2,xxx@x.ru,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
3,xxx@x.ru,2017-02-14T00:00:00Z,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
4,xxx@x.ru,2017-01-31T00:00:00Z,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
5,xxx@x.ru,2016-10-23T00:00:00Z,imesh.com,"In June 2016, a cache of over 51 million user ..."
6,yyy@y.com,2020-01-03T00:00:00Z,azcentral.com,"At an unconfirmed date, online Arizona newspap..."
7,yyy@y.com,2020-05-28T00:00:00Z,wishbone.io,"In January 2020, the online poll website Wishb..."
8,yyy@y.com,2017-11-04T00:00:00Z,myheritage.com,"In October 2017, a customer database belonging..."
9,yyy@y.com,2021-02-11T00:00:00Z,forums.vkmonline.com,"At an unconfirmed date, the Russian-language m..."


### Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>